In [1]:
#pip install torch transformers PyMuPDF

In [3]:
import fitz  # PyMuPDF
from transformers import BertTokenizer, BertModel
from transformers import BertTokenizer, BertForQuestionAnswering
import torch
import nltk
#nltk.download("punkt")
from nltk.tokenize import sent_tokenize

# 1. PDF-Text extrahieren
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as pdf:
        for page in pdf:
            text += page.get_text()
    return text

# 2. Text in Chunks aufteilen (max 512 Tokens für BERT)
def split_text_into_chunks(text, tokenizer, max_length=512):
    sentences = sent_tokenize(text)  # Robuste Satzaufteilung
    chunks = []
    current_chunk = []
    current_length = 0

    for sentence in sentences:
        tokenized_length = len(tokenizer.tokenize(sentence, truncation=True, max_length=max_length))
        if current_length + tokenized_length <= max_length:
            current_chunk.append(sentence)
            current_length += tokenized_length
        else:
            chunks.append(" ".join(current_chunk))
            current_chunk = [sentence]
            current_length = tokenized_length
    if current_chunk:
        chunks.append(" ".join(current_chunk))
    return chunks

# 3. Tokenizer und Modell laden
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

# 4. Embeddings erzeugen
def generate_embeddings(text_chunks, tokenizer, model):
    embeddings = []
    for chunk in text_chunks:
        inputs = tokenizer(chunk, return_tensors="pt", truncation=True, padding=True, max_length=512)
        outputs = model(**inputs)
        cls_embedding = outputs.last_hidden_state[:, 0, :]  # CLS-Token-Embedding
        embeddings.append(cls_embedding.detach().numpy())
    return embeddings

# 5. Workflow ausführen
pdf_path = "Der kleine Schmied.pdf"
text = extract_text_from_pdf(pdf_path)
chunks = split_text_into_chunks(text, tokenizer)
embeddings = generate_embeddings(chunks, tokenizer, model)

# Optional: Ergebnisse ausgeben oder speichern
for i, embedding in enumerate(embeddings):
    print(f"Chunk {i + 1} Embedding Shape: {embedding.shape}")

Keyword arguments {'truncation': True, 'max_length': 512} not recognized.
Keyword arguments {'truncation': True, 'max_length': 512} not recognized.
Keyword arguments {'truncation': True, 'max_length': 512} not recognized.
Keyword arguments {'truncation': True, 'max_length': 512} not recognized.
Keyword arguments {'truncation': True, 'max_length': 512} not recognized.
Keyword arguments {'truncation': True, 'max_length': 512} not recognized.
Keyword arguments {'truncation': True, 'max_length': 512} not recognized.
Keyword arguments {'truncation': True, 'max_length': 512} not recognized.
Keyword arguments {'truncation': True, 'max_length': 512} not recognized.
Keyword arguments {'truncation': True, 'max_length': 512} not recognized.
Keyword arguments {'truncation': True, 'max_length': 512} not recognized.
Keyword arguments {'truncation': True, 'max_length': 512} not recognized.
Keyword arguments {'truncation': True, 'max_length': 512} not recognized.
Keyword arguments {'truncation': True,

Chunk 1 Embedding Shape: (1, 768)
Chunk 2 Embedding Shape: (1, 768)
Chunk 3 Embedding Shape: (1, 768)


In [4]:
import numpy as np

# Beispieldatenbank: SQLite
import sqlite3

# Verbindung erstellen
conn = sqlite3.connect("embeddings.db")
cursor = conn.cursor()

# Tabelle erstellen
cursor.execute("""
CREATE TABLE IF NOT EXISTS embeddings (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    chunk TEXT,
    embedding BLOB
)
""")
conn.commit()

# Embeddings speichern
for i, (chunk, embedding) in enumerate(zip(chunks, embeddings)):
    embedding_blob = np.array(embedding).tobytes()  # Konvertiere Embedding in BLOB
    cursor.execute("INSERT INTO embeddings (chunk, embedding) VALUES (?, ?)", (chunk, embedding_blob))

conn.commit()
cursor.close()
conn.close()

In [5]:
import sqlite3
import numpy as np
import faiss

# Verbindung zur SQLite-Datenbank
conn = sqlite3.connect("embeddings.db")
cursor = conn.cursor()

# Embeddings aus der Datenbank abrufen
cursor.execute("SELECT id, embedding FROM embeddings")
data = cursor.fetchall()

# Embeddings in numpy-Array konvertieren
ids = []
embeddings = []
for row in data:
    ids.append(row[0])
    embeddings.append(np.frombuffer(row[1], dtype=np.float32))
embeddings = np.vstack(embeddings)

# FAISS-Index erstellen
dimension = embeddings.shape[1]  # z. B. 768
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# Index speichern (optional)
faiss.write_index(index, "faiss_index.bin")

cursor.close()
conn.close()

In [7]:
from transformers import BertTokenizer, BertModel, GPT2Tokenizer, GPT2LMHeadModel
from transformers import BertTokenizer, BertForQuestionAnswering
import faiss
import sqlite3

# FAISS-Index laden
index = faiss.read_index("faiss_index.bin")

# Anfrage-Embedding mit BERT erstellen
query = "Wer ist der Protagonist der Geschichte?"
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

inputs = tokenizer(query, return_tensors="pt", truncation=True, padding=True, max_length=512)
query_embedding = model(**inputs).last_hidden_state.mean(dim=1).detach().numpy()

# Ähnliche Einträge im FAISS-Index finden
k = 5  # Anzahl der relevanten Ergebnisse
distances, indices = index.search(query_embedding, k)

# Ergebnisse aus SQLite abrufen
conn = sqlite3.connect("embeddings.db")
cursor = conn.cursor()

# Holen aller IDs aus der Datenbank
cursor.execute("SELECT id FROM embeddings")
ids = [row[0] for row in cursor.fetchall()]

relevant_chunks = []
for idx in indices[0]:
    cursor.execute("SELECT chunk FROM embeddings WHERE id=?", (ids[idx],))
    chunk = cursor.fetchone()
    if chunk:
        relevant_chunks.append(chunk[0])

# Kontext mit abgerufenen Chunks erstellen
context = "\n".join(relevant_chunks) + "\n\nFrage: Wer ist der Protagonist der Geschichte?\nAntwort:"

# GPT-2 laden
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")

# Kontext auf maximale Token-Länge begrenzen
input_ids = gpt2_tokenizer.encode(context, return_tensors="pt", truncation=True, max_length=512)

# GPT-2 Antwort generieren
output = gpt2_model.generate(
    input_ids,
    max_new_tokens=100,
    num_return_sequences=1,
    no_repeat_ngram_size=2,
    pad_token_id=gpt2_tokenizer.eos_token_id  # Explizite Angabe
)

response = gpt2_tokenizer.decode(output[0], skip_special_tokens=True)

cursor.close()
conn.close()

print("Antwort von GPT-2:", response)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Antwort von GPT-2: Das 
hätte den Werkstoff wertlos gemacht 
An Samstagen wurde die Schmiede aufgeräumt und gefegt Jede Zange, jeder Hammer oder 
andere Geräte mussten an ihren Platz, das war Gesetz Und jedes Teilchen, sei es auch noch so 
klein, wurde aufgehoben „Ihr werdet schon sehen, für was das noch gut ist.“, erklärte der 
Großvater 
In der Vorweihnachtszeit, brachten die Landwirte ihre Gerätschaften zum Überprüften, bevor sie 
in den Winterschlaf versetzt wurden Pflugschare, Eggen, Platten, Kreuzhacken, Maurerhämmer 
mussten geschärft und gerichtet werden Wichtig war eine sorgfältige Kennzeichnung, damit 
jedes Teil seinem Besitzer zugeordnet werden konnte 
Eines Tages sagte der alte Schmied: „ Nun schmieden wir für jeden von euch einen 
Nussknacker“ Die Jungs schauten sich ungläubig an und verfolgten jeden Handgriff ihres 
Großvaters, der als erstes den passenden Stahl aussuchte Der Nussknacker durfte sich nicht 
verbiegen, sollte aber auch nicht zu schwer werden Und schön auss

In [8]:
# FAISS-Index durchsuchen
distances, indices = index.search(query_embedding, k)

# Relevante Chunks aus SQLite abrufen
relevant_chunks = []
for idx in indices[0]:
    cursor.execute("SELECT chunk FROM embeddings WHERE id=?", (ids[idx],))
    chunk = cursor.fetchone()
    if chunk:
        relevant_chunks.append(chunk[0])

# BERT initialisieren
from transformers import BertTokenizer, BertForQuestionAnswering
import torch

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForQuestionAnswering.from_pretrained("bert-base-uncased")

# BERT verwenden, um die präziseste Antwort zu finden
best_answer = None
best_score = float("-inf")

for chunk in relevant_chunks:
    # Tokenisierung
    inputs = tokenizer(
        query,  # Frage
        chunk,  # Kontext
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=512
    )

    # Vorhersage mit BERT
    with torch.no_grad():
        outputs = model(**inputs)
        start_scores = outputs.start_logits
        end_scores = outputs.end_logits

    # Score berechnen
    score = torch.max(start_scores) + torch.max(end_scores)

    # Beste Antwort auswählen
    if score > best_score:
        best_score = score
        start_index = torch.argmax(start_scores)
        end_index = torch.argmax(end_scores)
        best_answer = tokenizer.decode(inputs["input_ids"][0][start_index:end_index + 1])

# Ergebnis ausgeben
print("Beste Antwort:", best_answer)


ProgrammingError: Cannot operate on a closed cursor.